# Stroke Prediction Model Comparison

In [ ]:
# Importing Libraries
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
# EDA

In [ ]:
warnings.filterwarnings('ignore')

try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')
    
sns.set_palette('husl')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')

print(df.shape)
df.head()

In [ ]:
df.info()

In [ ]:
# Statistical summary
df.describe()

In [ ]:
print("Categorical Features:")
df.describe(include='object')

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Percentage': missing_percentage
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print(missing_df)

In [ ]:
print(f"Number of 'N/A' values in BMI: {(df['bmi'] == 'N/A').sum()}")
print(f"Data type of BMI column: {df['bmi'].dtype}")

In [ ]:
print("Duplicate Rows Analysis:")
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
print(f"Percentage of duplicates: {(duplicates/len(df))*100:.2f}%")

In [ ]:
print("Data Types:")
print(df.dtypes)

In [ ]:
# Analyze target variable distribution
print("Target Variable (Stroke) Distribution:")
stroke_counts = df['stroke'].value_counts()
stroke_percentage = df['stroke'].value_counts(normalize=True) * 100

target_df = pd.DataFrame({
    'Count': stroke_counts,
    'Percentage': stroke_percentage
})
print(target_df)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Count plot
sns.countplot(data=df, x='stroke', ax=axes[0], palette='Set2')
axes[0].set_title('Stroke Cases Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Stroke (0=No, 1=Yes)', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
for container in axes[0].containers:
    axes[0].bar_label(container)

# Pie chart
colors = ['#90EE90', '#FF6B6B']
axes[1].pie(stroke_counts, labels=['No Stroke', 'Stroke'], autopct='%1.1f%%', 
            startangle=90, colors=colors, explode=(0, 0.1))
axes[1].set_title('Stroke Cases Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Class Imbalance Detected: {stroke_percentage[0]:.2f}% ")
print("No Stroke vs {stroke_percentage[1]:.2f}% Stroke")

In [ ]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('id')
numerical_cols.remove('stroke')
print("Numerical Features:", numerical_cols)

In [ ]:
# Distribution plots for numerical features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    if col == 'bmi':
        data = pd.to_numeric(df[col], errors='coerce')
    else:
        data = df[col]
    
    axes[idx].hist(data.dropna(), bins=30, edgecolor='black', alpha=0.7, color='skyblue')
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)

if len(numerical_cols) < 6:
    for idx in range(len(numerical_cols), 6):
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Box plots for numerical features
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    if col == 'bmi':
        data = pd.to_numeric(df[col], errors='coerce')
    else:
        data = df[col]
    
    axes[idx].boxplot(data.dropna(), vert=True, patch_artist=True,
                     boxprops=dict(facecolor='lightblue', alpha=0.7),
                     medianprops=dict(color='red', linewidth=2))
    axes[idx].set_title(f'Box Plot of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel(col, fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)

if len(numerical_cols) < 6:
    for idx in range(len(numerical_cols), 6):
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print("Categorical Features:", categorical_cols)

for col in categorical_cols:
    print(f"\n{col}:")
    value_counts = df[col].value_counts()
    percentage = df[col].value_counts(normalize=True) * 100
    result_df = pd.DataFrame({
        'Count': value_counts,
        'Percentage': percentage
    })
    print(result_df)

In [ ]:
# Visualize categorical features
fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.ravel()

for idx, col in enumerate(categorical_cols):
    value_counts = df[col].value_counts()
    axes[idx].bar(range(len(value_counts)), value_counts.values, 
                  color=sns.color_palette('Set3', len(value_counts)), edgecolor='black')
    axes[idx].set_xticks(range(len(value_counts)))
    axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Count', fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)
    
    for i, v in enumerate(value_counts.values):
        axes[idx].text(i, v + 100, str(v), ha='center', va='bottom', fontweight='bold')

if len(categorical_cols) < 6:
    for idx in range(len(categorical_cols), 6):
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
df_temp = df.copy()
df_temp['bmi'] = pd.to_numeric(df_temp['bmi'], errors='coerce')

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for idx, col in enumerate(numerical_cols):
    sns.boxplot(data=df_temp, x='stroke', y=col, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{col} vs Stroke', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Stroke (0=No, 1=Yes)', fontsize=10)
    axes[idx].set_ylabel(col, fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)

if len(numerical_cols) < 6:
    for idx in range(len(numerical_cols), 6):
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Statistical comparison between stroke and no-stroke groups
print("Mean Values Comparison by Stroke Status:")
comparison = df_temp.groupby('stroke')[numerical_cols].mean()
print(comparison)
print("\nMedian Values Comparison by Stroke Status:")
comparison_median = df_temp.groupby('stroke')[numerical_cols].median()
print(comparison_median)

In [ ]:
# Categorical features vs Stroke
fig, axes = plt.subplots(3, 2, figsize=(16, 14))
axes = axes.ravel()

for idx, col in enumerate(categorical_cols):
    ct = pd.crosstab(df[col], df['stroke'], normalize='index') * 100
    
    ct.plot(kind='bar', ax=axes[idx], color=['#90EE90', '#FF6B6B'], 
            edgecolor='black', alpha=0.8)
    axes[idx].set_title(f'Stroke Rate by {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Percentage (%)', fontsize=10)
    axes[idx].legend(['No Stroke', 'Stroke'], loc='best')
    axes[idx].grid(axis='y', alpha=0.3)
    axes[idx].tick_params(axis='x', rotation=45)

if len(categorical_cols) < 6:
    for idx in range(len(categorical_cols), 6):
        axes[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Detailed stroke rate by categorical features
print("Stroke Rate by Categorical Features:")

for col in categorical_cols:
    print(f"\n{col}:")
    ct = pd.crosstab(df[col], df['stroke'], margins=True)
    ct_pct = pd.crosstab(df[col], df['stroke'], normalize='index') * 100
    
    result = pd.DataFrame({
        'No Stroke Count': ct[0][:-1],
        'Stroke Count': ct[1][:-1],
        'Total': ct['All'][:-1],
        'Stroke Rate (%)': ct_pct[1]
    })
    print(result)

In [ ]:
# Calculate correlation matrix for numerical features
correlation_cols = numerical_cols + ['stroke']
correlation_matrix = df_temp[correlation_cols].corr()

print("Correlation Matrix:")
print(correlation_matrix)

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', linewidths=1, square=True, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap of Numerical Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation with target variable
print("\nCorrelation with Stroke (Target Variable):")
stroke_correlation = correlation_matrix['stroke'].sort_values(ascending=False)
print(stroke_correlation)

plt.figure(figsize=(10, 6))
stroke_correlation[stroke_correlation.index != 'stroke'].plot(kind='barh', color='teal', edgecolor='black')
plt.title('Feature Correlation with Stroke', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.axvline(x=0, color='red', linestyle='--', linewidth=1)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("Outlier Detection (IQR Method):")

outlier_summary = {}

for col in numerical_cols:
    if col == 'bmi':
        data = pd.to_numeric(df[col], errors='coerce').dropna()
    else:
        data = df[col].dropna()
    
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data < lower_bound) | (data > upper_bound)]
    outlier_count = len(outliers)
    outlier_percentage = (outlier_count / len(data)) * 100
    
    outlier_summary[col] = {
        'Count': outlier_count,
        'Percentage': f"{outlier_percentage:.2f}%",
        'Lower Bound': f"{lower_bound:.2f}",
        'Upper Bound': f"{upper_bound:.2f}"
    }

outlier_df = pd.DataFrame(outlier_summary).T
print(outlier_df)

In [ ]:
print("KEY FINDINGS FROM EXPLORATORY DATA ANALYSIS")

print("1. DATASET OVERVIEW:")
print(f"   • Total Records: {len(df):,}")
print(f"   • Total Features: {len(df.columns)}")
print(f"   • Numerical Features: {len(numerical_cols)}")
print(f"   • Categorical Features: {len(categorical_cols)}")

print("\n2. TARGET VARIABLE (STROKE):")
stroke_pct = (df['stroke'].sum() / len(df)) * 100
print(f"   • Stroke Cases: {df['stroke'].sum():,} ({stroke_pct:.2f}%)")
print(f"   • No Stroke Cases: {len(df) - df['stroke'].sum():,} ({100-stroke_pct:.2f}%)")
print(f"   • Highly Imbalanced Dataset - Will need handling")

print("\n3. DATA QUALITY:")
bmi_missing = (df['bmi'] == 'N/A').sum()
print(f"   • BMI Missing Values: {bmi_missing} ({(bmi_missing/len(df))*100:.2f}%)")
print(f"   • Duplicate Rows: {df.duplicated().sum()}")
print(f"   • BMI stored as object (needs conversion)")

print("\n4. KEY CORRELATIONS WITH STROKE:")
top_corr = stroke_correlation[stroke_correlation.index != 'stroke'].head(3)
for feature, corr_value in top_corr.items():
    print(f"   • {feature}: {corr_value:.3f}")

print("\n5. IMPORTANT OBSERVATIONS:")
print("   • Age shows strongest correlation with stroke")
print("   • Glucose level also shows positive correlation")
print("   • Hypertension and heart disease are binary indicators")
print("   • Gender, marital status, work type, and residence need encoding")
print("   • Smoking status has multiple categories including 'Unknown'")

In [ ]:
#Data Cleaning, Preprocessing & Encoding

In [ ]:
# Create a copy for preprocessing
df_processed = df.copy()

print("STEP 1: Handling Missing Values")

# Convert BMI from object to float, replacing 'N/A' with NaN
print("\nConverting BMI column from object to numeric...")
df_processed['bmi'] = pd.to_numeric(df_processed['bmi'], errors='coerce')
print(f"BMI data type after conversion: {df_processed['bmi'].dtype}")
print(f"BMI missing values: {df_processed['bmi'].isnull().sum()}")

# Impute missing BMI values with median (robust to outliers)
bmi_median = df_processed['bmi'].median()
print(f"\nImputing {df_processed['bmi'].isnull().sum()} missing BMI values with median: {bmi_median:.2f}")
df_processed['bmi'].fillna(bmi_median, inplace=True)

print(f"\nAfter imputation - BMI missing values: {df_processed['bmi'].isnull().sum()}")
print("Missing values handled successfully!")

In [ ]:
print("\nSTEP 2: Removing Unnecessary Columns")

# Remove ID column (not useful for prediction)
print("\nRemoving 'id' column (not useful for prediction)...")
df_processed = df_processed.drop('id', axis=1)

print(f"Columns after removal: {list(df_processed.columns)}")
print(f"Shape after removal: {df_processed.shape}")
print("Unnecessary columns removed!")

In [ ]:
print("\nSTEP 3: Handling Duplicate Rows")

duplicates_before = df_processed.duplicated().sum()
print(f"Duplicate rows before removal: {duplicates_before}")

if duplicates_before > 0:
    df_processed = df_processed.drop_duplicates()
    print(f"Duplicate rows after removal: {df_processed.duplicated().sum()}")
    print(f"Rows removed: {duplicates_before}")
    print("Duplicates removed!")
else:
    print("No duplicates found!")

print(f"\nFinal shape: {df_processed.shape}")

In [ ]:
from sklearn.preprocessing import LabelEncoder

print("STEP 4: Encoding Categorical Variables")

categorical_columns = df_processed.select_dtypes(include=['object']).columns.tolist()
print(f"Categorical columns to encode: {categorical_columns}")

# Initialize label encoder
label_encoders = {}

# Encode each categorical column
for col in categorical_columns:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col])
    label_encoders[col] = le
    
    print(f"\n{col}:")
    print(f"  Original categories: {list(le.classes_)}")
    print(f"  Encoded values: {list(range(len(le.classes_)))}")

print("\nAll categorical variables encoded successfully!")

In [ ]:
print("STEP 5: Verification of Processed Data")

print("\nProcessed Dataset Info:")
df_processed.info()

print("Processed Dataset - First 5 Rows:")
display(df_processed.head())

print("Statistical Summary of Processed Data:")
display(df_processed.describe())

print("Missing Values After Preprocessing:")
print(df_processed.isnull().sum())

print("\nData preprocessing completed successfully!")
print(f"Final dataset shape: {df_processed.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler

print("STEP 6: Feature Scaling")

# Separate features and target
X = df_processed.drop('stroke', axis=1)
y = df_processed['stroke']

print(f"\nFeatures shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")

# Initialize scaler
scaler = StandardScaler()

# Fit and transform features
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame for better readability
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("\nFeatures scaled using StandardScaler!")
print("\nScaled Features - First 5 rows:")
display(X_scaled_df.head())

print("\nScaled Features Statistics:")
display(X_scaled_df.describe())

In [ ]:
# Model Building & Evaluation

In [ ]:
from sklearn.model_selection import train_test_split

print("Train-Test Split")

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nOriginal Dataset Size: {len(X_scaled)}")
print(f"Training Set Size: {len(X_train)} ({(len(X_train)/len(X_scaled))*100:.1f}%)")
print(f"Testing Set Size: {len(X_test)} ({(len(X_test)/len(X_scaled))*100:.1f}%)")

print("\nTraining Set Distribution:")
print(f"  No Stroke (0): {(y_train == 0).sum()}")
print(f"  Stroke (1): {(y_train == 1).sum()}")

print("\nTesting Set Distribution:")
print(f"  No Stroke (0): {(y_test == 0).sum()}")
print(f"  Stroke (1): {(y_test == 1).sum()}")

print("\nTrain-Test split completed with stratification!")

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)
import time


In [ ]:
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'AdaBoost': AdaBoostClassifier(random_state=42, algorithm='SAMME'),
    'XGBoost': XGBClassifier(random_state=42, n_jobs=-1, eval_metric='logloss')
}

results = {}
print("TRAINING AND EVALUATING MACHINE LEARNING MODELS")
print(f"\nTotal models to train: {len(models)}")

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    
    # Start timer
    start_time = time.time()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculate training time
    training_time = time.time() - start_time
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    
    # Calculate ROC-AUC
    if y_pred_proba is not None:
        roc_auc = roc_auc_score(y_test, y_pred_proba)
    else:
        roc_auc = None
    
    results[model_name] = {
        'Model': model,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC-AUC': roc_auc,
        'Training Time': training_time,
        'Predictions': y_pred,
        'Prediction Probabilities': y_pred_proba
    }
    
    print(f"\n {model_name} Training Completed!")
    print(f"   Training Time: {training_time:.2f} seconds")
    print(f"\n   Performance Metrics:")
    print(f"   Accuracy:  {accuracy*100:.2f}%")
    print(f"   Precision: {precision*100:.2f}%")
    print(f"   Recall:    {recall*100:.2f}%")
    print(f"   F1-Score:  {f1*100:.2f}%")
    if roc_auc is not None:
        print(f"   ROC-AUC:   {roc_auc:.4f}")

print("All models trained and evaluated successfully!")

In [ ]:
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy (%)': [results[model]['Accuracy'] * 100 for model in results],
    'Precision (%)': [results[model]['Precision'] * 100 for model in results],
    'Recall (%)': [results[model]['Recall'] * 100 for model in results],
    'F1-Score (%)': [results[model]['F1-Score'] * 100 for model in results],
    'ROC-AUC': [results[model]['ROC-AUC'] if results[model]['ROC-AUC'] else 0 for model in results],
    'Training Time (s)': [results[model]['Training Time'] for model in results]
})

comparison_df = comparison_df.sort_values('Accuracy (%)', ascending=False).reset_index(drop=True)

print("MODEL COMPARISON SUMMARY")
print("\n")
display(comparison_df)

best_model_name = comparison_df.iloc[0]['Model']
best_accuracy = comparison_df.iloc[0]['Accuracy (%)']

print(f"\n{'='*80}")
print(f"BEST MODEL: {best_model_name}")
print(f"Accuracy: {best_accuracy:.2f}%")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

#Accuracy Comparison
axes[0, 0].barh(comparison_df['Model'], comparison_df['Accuracy (%)'], color='steelblue', edgecolor='black')
axes[0, 0].set_xlabel('Accuracy (%)', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0, 0].grid(axis='x', alpha=0.3)
for i, v in enumerate(comparison_df['Accuracy (%)']):
    axes[0, 0].text(v + 0.5, i, f'{v:.2f}%', va='center', fontweight='bold')

#Precision, Recall, F1-Score Comparison
x = np.arange(len(comparison_df))
width = 0.25
axes[0, 1].bar(x - width, comparison_df['Precision (%)'], width, label='Precision', color='#FF6B6B', edgecolor='black')
axes[0, 1].bar(x, comparison_df['Recall (%)'], width, label='Recall', color='#4ECDC4', edgecolor='black')
axes[0, 1].bar(x + width, comparison_df['F1-Score (%)'], width, label='F1-Score', color='#95E1D3', edgecolor='black')
axes[0, 1].set_xlabel('Models', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Score (%)', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Precision, Recall & F1-Score Comparison', fontsize=14, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(comparison_df['Model'], rotation=45, ha='right')
axes[0, 1].legend()
axes[0, 1].grid(axis='y', alpha=0.3)

#ROC-AUC Comparison
axes[1, 0].barh(comparison_df['Model'], comparison_df['ROC-AUC'], color='coral', edgecolor='black')
axes[1, 0].set_xlabel('ROC-AUC Score', fontsize=12, fontweight='bold')
axes[1, 0].set_title('ROC-AUC Score Comparison', fontsize=14, fontweight='bold')
axes[1, 0].set_xlim([0, 1])
axes[1, 0].grid(axis='x', alpha=0.3)
for i, v in enumerate(comparison_df['ROC-AUC']):
    axes[1, 0].text(v + 0.02, i, f'{v:.4f}', va='center', fontweight='bold')

#Training Time Comparison
axes[1, 1].bar(comparison_df['Model'], comparison_df['Training Time (s)'], color='mediumpurple', edgecolor='black')
axes[1, 1].set_ylabel('Time (seconds)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Training Time Comparison', fontsize=14, fontweight='bold')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(comparison_df['Training Time (s)']):
    axes[1, 1].text(i, v + 0.01, f'{v:.2f}s', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
n_models = len(results)
n_rows = (n_models + 2) // 3

# Plot confusion matrices
fig, axes = plt.subplots(n_rows, 3, figsize=(18, 6 * n_rows))
if n_rows == 1:
    axes = axes.reshape(1, -1)
axes = axes.ravel()

for idx, (model_name, result) in enumerate(results.items()):
    cm = confusion_matrix(y_test, result['Predictions'])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, ax=axes[idx],
                xticklabels=['No Stroke', 'Stroke'], yticklabels=['No Stroke', 'Stroke'])
    axes[idx].set_title(f'{model_name}\nAccuracy: {result["Accuracy"]*100:.2f}%', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Actual', fontsize=10)
    axes[idx].set_xlabel('Predicted', fontsize=10)

for idx in range(n_models, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
print("DETAILED CLASSIFICATION REPORTS")

for model_name, result in results.items():
    print(f"\n{model_name}")
    print(classification_report(y_test, result['Predictions'], 
                                target_names=['No Stroke', 'Stroke'],
                                digits=4))

In [ ]:
# Plot ROC curves
plt.figure(figsize=(12, 8))

colors = ['blue', 'red', 'green', 'orange', 'purple']

for idx, (model_name, result) in enumerate(results.items()):
    if result['Prediction Probabilities'] is not None:
        fpr, tpr, _ = roc_curve(y_test, result['Prediction Probabilities'])
        roc_auc = result['ROC-AUC']
        
        plt.plot(fpr, tpr, color=colors[idx % len(colors)], lw=2, 
                label=f'{model_name} (AUC = {roc_auc:.4f})')

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random Classifier')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12, fontweight='bold')
plt.ylabel('True Positive Rate', fontsize=12, fontweight='bold')
plt.title('Receiver Operating Characteristic (ROC) Curves', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=10)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Plot feature importance for tree-based models
tree_models = ['Random Forest', 'Decision Tree', 'AdaBoost','XGBoost']
available_tree_models = [m for m in tree_models if m in results]

if len(available_tree_models) > 0:
    n_cols = 2
    n_rows = (len(available_tree_models) + 1) // 2
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 6 * n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    axes = axes.ravel()

    for idx, model_name in enumerate(available_tree_models):
        model = results[model_name]['Model']
        
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            feature_names = X.columns
            importance_df = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values('Importance', ascending=False)
            axes[idx].barh(importance_df['Feature'], importance_df['Importance'], 
                          color='teal', edgecolor='black')
            axes[idx].set_xlabel('Importance', fontsize=10, fontweight='bold')
            axes[idx].set_title(f'{model_name} - Feature Importance', 
                               fontsize=12, fontweight='bold')
            axes[idx].grid(axis='x', alpha=0.3)
    
    for idx in range(len(available_tree_models), len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("No tree-based models available for feature importance visualization.")

In [ ]:
print("FINAL SUMMARY & RECOMMENDATIONS")
best_model_name = comparison_df.iloc[0]['Model']
best_result = results[best_model_name]

print(f"\nBEST PERFORMING MODEL: {best_model_name}")
print(f"   Accuracy:      {best_result['Accuracy']*100:.2f}%")
print(f"   Precision:     {best_result['Precision']*100:.2f}%")
print(f"   Recall:        {best_result['Recall']*100:.2f}%")
print(f"   F1-Score:      {best_result['F1-Score']*100:.2f}%")
if best_result['ROC-AUC']:
    print(f"   ROC-AUC:       {best_result['ROC-AUC']:.4f}")
print(f"   Training Time: {best_result['Training Time']:.2f} seconds")